# 引入所需要模块

In [1]:
import os
import random
import pandas as pd
import numpy as np
from PIL import Image

# 加载数据集

In [3]:
root_dir=os.path.abspath('.')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [4]:
#随机读取图片
i=random.choice(train.index)
img_name=train.ID[i]
print(img_name)
img=Image.open(os.path.join('Train',img_name))
img.show()
print(train.Class[i])

7611.jpg
YOUNG


# 格式化图片尺寸和将数据存到numpy数组

In [5]:
temp=[]
for img_name in train.ID:
    img_path=os.path.join('Train',img_name)
    img=Image.open(img_path)
    img=img.resize((32,32))
    array=np.array(img)
    temp.append(array.astype('float32'))
train_x=np.stack(temp)
print(train_x.shape)
print(train_x.ndim)

(19906, 32, 32, 3)
4


In [6]:
temp=[]
for img_name in test.ID:
    img_path=os.path.join('Test',img_name)
    img=Image.open(img_path)
    img=img.resize((32,32))
    array=np.array(img)
    temp.append(array.astype('float32'))
test_x=np.stack(temp)
print(test_x.shape)

(6636, 32, 32, 3)


In [7]:
train_x = train_x / 255.
test_x = test_x / 255.

In [8]:
train.Class.value_counts(normalize=True)

MIDDLE    0.542751
YOUNG     0.336883
OLD       0.120366
Name: Class, dtype: float64

In [9]:
test['Class'] = 'MIDDLE'
test.to_csv('sub01.csv', index=False)

# 创建模型

In [10]:
import keras
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
train_y=lb.fit_transform(train.Class)
train_y=keras.utils.np_utils.to_categorical(train_y)
print(train_y)
print(train_y.shape)

Using TensorFlow backend.


[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 ..., 
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]
(19906, 3)


In [11]:
#构建神经网络
input_num_units=(32,32,3)
hidden_num_units=500
output_num_units=3
epochs=5
batch_size=128

In [36]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,InputLayer
model=Sequential({
    InputLayer(input_shape=input_num_units),
    Flatten(),
    Dense(units=hidden_num_units,activation='relu'),
    Dense(input_shape=(32,32,3),units=output_num_units,activation='softmax')
})
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense_45 (Dense)             (None, 500)               1536500   
_________________________________________________________________
dense_46 (Dense)             (None, 3)                 1503      
Total params: 1,538,003
Trainable params: 1,538,003
Non-trainable params: 0
_________________________________________________________________


# 编译模型

In [37]:
# model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/5
19906/19906 [==============================] - 4s - loss: 0.8878 - acc: 0.5809     
Epoch 2/5
19906/19906 [==============================] - 4s - loss: 0.8420 - acc: 0.6077     
Epoch 3/5
19906/19906 [==============================] - 4s - loss: 0.8210 - acc: 0.6214     
Epoch 4/5
19906/19906 [==============================] - 4s - loss: 0.8149 - acc: 0.6194     
Epoch 5/5
19906/19906 [==============================] - 4s - loss: 0.8042 - acc: 0.6305     


In [38]:
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1, validation_split=0.2)

Train on 15924 samples, validate on 3982 samples
Epoch 1/5
15924/15924 [==============================] - 3s - loss: 0.7970 - acc: 0.6375 - val_loss: 0.7854 - val_acc: 0.6396
Epoch 2/5
15924/15924 [==============================] - 3s - loss: 0.7919 - acc: 0.6378 - val_loss: 0.7767 - val_acc: 0.6519
Epoch 3/5
15924/15924 [==============================] - 3s - loss: 0.7870 - acc: 0.6404 - val_loss: 0.7754 - val_acc: 0.6534
Epoch 4/5
15924/15924 [==============================] - 3s - loss: 0.7806 - acc: 0.6439 - val_loss: 0.7715 - val_acc: 0.6524
Epoch 5/5
15924/15924 [==============================] - 3s - loss: 0.7755 - acc: 0.6519 - val_loss: 0.7970 - val_acc: 0.6346


# 输出结果

In [39]:
pred=model.predict_classes(test_x)
pred=lb.inverse_transform(pred)
print(pred)
test['Class']=pred
test.to_csv('sub02.csv',index=False)

6432/6636 [============================>.] - ETA: 0s['YOUNG' 'YOUNG' 'MIDDLE' ..., 'MIDDLE' 'MIDDLE' 'YOUNG']


In [48]:
i = random.choice(train.index)
img_name = train.ID[i]

img=Image.open(os.path.join('Train',img_name))
img.show()
pred = model.predict_classes(train_x)
print('Original:', train.Class[i], 'Predicted:', lb.inverse_transform(pred[i]))

19584/19906 [============================>.] - ETA: 0sOriginal: YOUNG Predicted: MIDDLE
